In [1]:
%%HTML
<style type="text/css">

div.h1 {
    font-size: 32px; 
    margin-bottom:2px;
    background-color: steelblue; 
    color: white; 
    text-align: center;
}
div.h2 {
    background-color: steelblue; 
    color: white; 
    padding: 8px; 
    padding-right: 300px; 
    font-size: 24px; 
    max-width: 1500px; 
    margin-top: 50px;
    margin-bottom:4px;
    
}
div.h3 {
    color: steelblue; 
    font-size: 20px; 
    margin-top: 4px; 
    margin-bottom:8px;
}
div.h4 {
    font-size: 15px; 
    margin-top: 20px; 
    margin-bottom: 8px;
}

</style>


<div class="h1">30 Days of ML- Competition</div>



This notebook is created as part  of 30 day ML challenge. This book is divided in two part EDA & kfold(https://www.kaggle.com/yogidsba/eda-kfold-30-day-ml) and Modelbuilding

**About the data**
The dataset is used for this competition is synthetic, but based on a real dataset. The original dataset deals with predicting the amount of an insurance claim. Although the features are anonymized, they have properties relating to real-world features.


**Evaluation**

Submissions are scored on the root mean squared error.


**Special mention**
I learned about kfold and finding mean of prediction from Abhishek Thakur YoutubeChannel. For that code you can refer his notebook or watch youtube channel.Thank you Abhisek

<div class="h1">Table of Contents</div>

  - <a href='#il'>Import Libraries</a>
  - <a href='#rd'>Read and Understand Data</a>
  - <a href='#mb'>Model Building </a>
 

<a id='il'></a>
<div class="h3">Import Libraries</div>

In [2]:
# Familiar imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
 # For ordinal encoding categorical variables, splitting data
from sklearn.preprocessing import OrdinalEncoder

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
print("loaded libraries")

loaded libraries


<a id='rd'></a>
<div class="h3">Read and Understand Data</div>

Next, we'll load the train_kfold csv that was created before and has additional columns containing kfold and test data.  


In [3]:
# Load the training data
train = pd.read_csv("../input/30days-folds/train_folds.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("/kaggle/input/30-days-of-ml/sample_submission.csv")
# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
0,1,B,B,B,C,B,B,A,E,C,...,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,0
1,2,B,B,A,A,B,D,A,F,A,...,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,2
2,3,A,A,A,C,B,D,A,D,A,...,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4
3,4,B,B,A,C,B,D,A,E,C,...,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,3
4,6,A,A,A,C,B,D,A,E,A,...,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,1


There are 10 categorical variables which we will have to encode 

In [4]:
#creating a list of categorical variables
#cat_col=train.select_dtypes(include='object').columns.to_list()
cat_col = [col for col in train.columns if 'cat' in col]

cat_col

['cat0',
 'cat1',
 'cat2',
 'cat3',
 'cat4',
 'cat5',
 'cat6',
 'cat7',
 'cat8',
 'cat9']

In [5]:
#get only usefulfeatures
useful_features=[col for col in train.columns if col not in ['id','target','kfold']]

# List of categorical columns

test=test[useful_features]


# for col in cat_col:
#     train[f"cont_{col}"] = train.groupby(col)[col].transform("count")
#     test[f"cont_{col}"] = test.groupby(col)[col].transform("count")

    
# useful_features = [c for c in train.columns if c not in ("id", "target", "kfold")]
# cat_col = [col for col in useful_features if col.startswith("cat")]

<a id='xg'></a>
<div class="h3">Understanding Hyperparameters for XGboost</div>

- **njobs**
    - To make use of all cpu
- **tree_method='gpu_hist'**
    - To use Gpu and runs faster
- **n_estimators**
    - The number of sequential trees to be modeled.
    - Though XGB is fairly robust at higher number of trees but it can still overfit at a         point. Hence, this should be tuned using CV for a particular learning rate.
- **learning_rate**
    - This determines the impact of each tree on the final outcome  XGB works by starting with an initial estimate which is updated using the output of each tree. The learning parameter controls the magnitude of this change in the estimates.
    - Lower values are generally preferred as they make the model robust to the specific characteristics of tree and thus allowing it to generalize well.
    - Lower values would require higher number of trees to model all the relations and will be computationally expensive.

- **colsample_bytree**
    -  Denotes the fraction of columns to be randomly  samples for each tree.
    -  Typical values: 0.5-1
- **reg_alpha**
    - L1 regularization term on weight (analogous to Lasso regression)
    - Can be used in case of very high dimensionality so that the algorithm runs faster           when implemented

- **subsample**
    -  Denotes the fraction of observations to be randomly samples for each tree.
    - Lower values make the algorithm more conservative and prevents overfitting but too        small values might lead to under-fitting.
    - Typical values: 0.5-1

- **max_depth**
     - Used to control over-fitting as higher depth will allow model to learn relations          very  specific to a particular sample.
    - Should be tuned using CV.
    - Typical values: 3-10
- **gamma**
    - A node is split only when the resulting split gives a positive reduction in the loss       function. 
    - Gamma specifies the minimum loss reduction required to make a split.
    - Makes the algorithm conservative. The values can vary depending on the loss function      and should be tuned.
    
- **eval_metric**
    - The metric to be used for validation data.
    - The default values are rmse for regression and error for classification.
    - Typical values are:
        - rmse – root mean square error
        - mae – mean absolute error
        - logloss – negative log-likelihood
        - error – Binary classification error rate (0.5 threshold)
        - merror – Multiclass classification error rate
        - mlogloss – Multiclass logloss
        - auc: Area under the curve

In [6]:

final_pred=[] #save your predictions in this list
for fold in range(5):
    
    xtrain=train[train.kfold!=fold].reset_index(drop=True) # get kfold which is not equal to fold
    xvalid=train[train.kfold==fold].reset_index(drop=True) # K is equal to fold
    
    ytrain=xtrain.target
    yvalid=xvalid.target
    xtest= test.copy()
 
    xtrain=xtrain[useful_features]
    xvalid=xvalid[useful_features]

    ordinal_encoder = OrdinalEncoder()
    xtrain[cat_col] = ordinal_encoder.fit_transform(xtrain[cat_col])
    xvalid[cat_col] = ordinal_encoder.transform(xvalid[cat_col])
    xtest[cat_col] = ordinal_encoder.transform(xtest[cat_col])
    
    model = XGBRegressor(random_state=fold, 
                         n_jobs=4,
                         n_estimators= 5000,
                         tree_method='gpu_hist',
                         learning_rate= 0.01,
                         subsample= 0.9,
                         max_depth= 5,
                         colsample_bytree= 0.5,
                         reg_alpha = 30,eval_metric='rmse')
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_pred.append(test_preds)
    print(fold, mean_squared_error(yvalid, preds_valid, squared=False))
    


0 0.718835963243177
1 0.7184024786516624
2 0.7204346624160692
3 0.7203607457429835
4 0.7192133316185308


In [7]:

#get the mean of all the predictions.
preds = np.mean(np.column_stack(final_pred), axis=1)
sample_submission.target = preds
sample_submission.to_csv("submission.csv", index=False)